# Train network notebook

This notebook is coded to help train a U-net using atrous (dilated) convolutions.
It can be used either in GUI mode with jupyter or in console mode (TO CHECK).

First, we import the modules.
We also import a set of configuration functions that help us when manipulation config files, which are the backbone of the network.

In [11]:
import json
import os
import time
from AxonDeepSeg.config_tools import *

We define the name of the directory we are going to save our model to.

In [12]:
dir_model_save='no_dilation_without_bn'

We also define the configuration of the model.

In [13]:
default_configuration()

{'balanced_weights': [1.1, 1, 1.3],
 'batch_norm_activate': True,
 'batch_norm_decay_decay_activate': True,
 'batch_norm_decay_decay_period': 24000,
 'batch_norm_decay_ending_decay': 0.9,
 'batch_norm_decay_starting_decay': 0.7,
 'batch_size': 8,
 'convolution_per_layer': [3, 3, 3, 3],
 'da-elastic-activate': True,
 'da-elastic-alpha_max': 9,
 'da-elastic-order': 3,
 'da-flipping-activate': True,
 'da-flipping-order': 4,
 'da-gaussian_blur-activate': True,
 'da-gaussian_blur-order': 6,
 'da-gaussian_blur-sigma_max': 1.5,
 'da-noise_addition-activate': False,
 'da-noise_addition-order': 5,
 'da-random_rotation-activate': False,
 'da-random_rotation-high_bound': 89,
 'da-random_rotation-low_bound': 5,
 'da-random_rotation-order': 2,
 'da-rescaling-activate': False,
 'da-rescaling-factor_max': 1.2,
 'da-rescaling-order': 1,
 'da-shifting-activate': True,
 'da-shifting-order': 0,
 'da-shifting-percentage_max': 0.1,
 'da-type': 'all',
 'dataset_mean': 120.95,
 'dataset_variance': 60.23,
 'd

# Dictionnary with the updates
update_dict={
    'dilation_rate': [[1, 1, 1], [1, 1, 1], [1,1,1,1,1,1]],
    'convolution_per_layer': [3, 3, 6],
    'features_per_convolution': [[[1, 16], [16, 16], [16, 16]],
  [[16, 32], [32, 32], [32, 32]],
  [[32, 64], [64, 64], [64, 64],[64, 128], [128, 128], [128, 128]]],
    'size_of_convolutions_per_layer': [[5, 5, 5],
  [3, 3, 3],
  [3, 3, 3, 3, 3, 3]]
}


config=update_config(default_configuration(),update_dict)

In [ ]:
# Dictionnary with the updates
update_dict={
    'batch_norm_activate':False
}


config=update_config(default_configuration(),update_dict)

In [5]:
config

{'balanced_weights': [1.1, 1, 1.3],
 'batch_norm_activate': True,
 'batch_norm_decay_decay_activate': True,
 'batch_norm_decay_decay_period': 24000,
 'batch_norm_decay_ending_decay': 0.9,
 'batch_norm_decay_starting_decay': 0.7,
 'batch_size': 8,
 'convolution_per_layer': [3, 3, 6],
 'da-elastic-activate': True,
 'da-elastic-alpha_max': 9,
 'da-elastic-order': 3,
 'da-flipping-activate': True,
 'da-flipping-order': 4,
 'da-gaussian_blur-activate': True,
 'da-gaussian_blur-order': 6,
 'da-gaussian_blur-sigma_max': 1.5,
 'da-noise_addition-activate': False,
 'da-noise_addition-order': 5,
 'da-random_rotation-activate': False,
 'da-random_rotation-high_bound': 89,
 'da-random_rotation-low_bound': 5,
 'da-random_rotation-order': 2,
 'da-rescaling-activate': False,
 'da-rescaling-factor_max': 1.2,
 'da-rescaling-order': 1,
 'da-shifting-activate': True,
 'da-shifting-order': 0,
 'da-shifting-percentage_max': 0.1,
 'da-type': 'all',
 'dataset_mean': 120.95,
 'dataset_variance': 60.23,
 'dept

We create variables to be used by the network when training as well as the directory to store the model. If the directory already exists we add a timestamp at the end of the directory name.

In [6]:
path_model = os.path.join('../models/', dir_model_save)
if os.path.exists(path_model): #If the path exists already we add a timestamp at the end of dir name.
    dir_model_save += '_' + str(int(time.time()))[-4:]
    path_model = os.path.join('../models/', dir_model_save)
os.makedirs(path_model)

We then make sure that the config file is valid by using a function we imported. __update_config__ updates the first config (default_configuration()) with the second (config_network) so that the parameters in config_network are at least the parameters needed by the network to run.

In [7]:
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config)

Management of the config file: if it already exists, we load it, else, we write it.

In [8]:
path_config_file=os.path.join(path_model,'config_network.json')
if os.path.exists(path_config_file): # if there is already a configfile
    with open(path_config_file, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_config_file, 'w') as f:
        json.dump(config, f, indent=2)
        config_network=config
   
print 'Chosen parameters :\n'
for param_name, param_value in config_network.iteritems():
    print param_name, ' - ', param_value

Chosen parameters :

trainingset_patchsize  -  512
thresholds  -  [0, 0.2, 0.8]
learning_rate_decay_type  -  polynomial
convolution_per_layer  -  [3, 3, 6]
da-gaussian_blur-sigma_max  -  1.5
da-shifting-percentage_max  -  0.1
da-shifting-activate  -  True
batch_norm_decay_decay_activate  -  True
da-flipping-activate  -  True
batch_norm_decay_starting_decay  -  0.7
n_classes  -  3
learning_rate_decay_activate  -  True
trainingset  -  SEM_3c_512
da-random_rotation-activate  -  False
da-rescaling-activate  -  False
batch_norm_decay_decay_period  -  24000
da-shifting-order  -  0
features_per_convolution  -  [[[1, 16], [16, 16], [16, 16]], [[16, 32], [32, 32], [32, 32]], [[32, 64], [64, 64], [64, 64], [64, 128], [128, 128], [128, 128]]]
batch_norm_decay_ending_decay  -  0.9
da-gaussian_blur-order  -  6
size_of_convolutions_per_layer  -  [[5, 5, 5], [3, 3, 3], [3, 3, 3, 3, 3, 3]]
da-gaussian_blur-activate  -  True
dataset_mean  -  120.95
dilation_rate  -  [[1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 

We indicate to the network where to find our trainingset

In [9]:
path_trainingset = os.path.join('../data/', config_network["trainingset"], 'training/')
path_model_init=None

Last part, but not the least: actually training the model!

In [10]:
from AxonDeepSeg.train_network import train_model
train_model(path_trainingset, path_model, config_network, path_model_init=path_model_init,gpu='/gpu:0')

('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 16])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [16, 16])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 2, 'Features: ', [16, 16])
('Size:', 5)
('Layer: ', 1, ' Conv: ', 0, 'Features: ', [16, 32])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 1, 'Features: ', [32, 32])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 2, 'Features: ', [32, 32])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 0, 'Features: ', [32, 64])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 1, 'Features: ', [64, 64])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 2, 'Features: ', [64, 64])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 3, 'Features: ', [64, 128])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 4, 'Features: ', [128, 128])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 5, 'Features: ', [128, 128])
('Size:', 3)


IndexError: list index out of range